In [1]:
import os
os.environ["PYSPARK_PYTHON"]="/usr/local/bin/ipython3"

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "final_project")

In [6]:
# read chat data to rdd
file_to_read = 'chat_example.txt'
rdd_chat = sc.textFile(file_to_read).cache()

In [7]:
# print example from rdd_chat
rdd_chat.take(10)    

['1/3/19, 8:37 PM - Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.',
 '1/3/19, 8:37 PM - Jorge U: Jorge',
 '1/3/19, 8:37 PM - Jorge U: 🙂',
 '1/3/19, 11:49 PM - Edwin Vargas: <Media omitted>',
 '1/3/19, 11:49 PM - Edwin Vargas: Salen muy buenas',
 '1/3/19, 11:50 PM - Edwin Vargas: Lo digo porque ya las he visto en acción',
 '1/3/19, 11:58 PM - Edwin Vargas: http://www.incolmotos-yamaha.com.co/site/productos/motos/urbanas-trabajo-y-deportivas/ybr-125',
 '1/3/19, 11:58 PM - Edwin Vargas: O esa',
 '1/5/19, 10:05 PM - Edwin Vargas: Wey',
 '1/5/19, 10:05 PM - Edwin Vargas: Qué va a hacer mañana?']

In [111]:
import re

# function to clean message from whatsapp chat
def clean_message(message):
    return clean_articles(clean_username(clean_date(clean_whatsapp_message(message))))

# function to clean date from message 
def clean_date(message):
    return re.sub(r'(\d+\/\d+\/\d+)(,)(\s)(\d+:\d+)(\s)(\w+)(\s)(-)', '', message)

# function to clean name from message
def clean_username(message):
    return re.sub(r'([\s\wñÑáéíóúÁÉÍÓÚ]+)(:)', '', message)

# fuction to clean articles from message
def clean_articles(message):
    return re.sub(r'(?i)(\s)+(\b)(el|los|la|las|un|unos|una|unas|a|de|del|al|lo|con|contra)(\b)', '', message)

# function to clean default whatsapp messages
def clean_whatsapp_message(message):
    return '' if('Messages to this chat and calls are now' in message or 'Las llamadas y mensajes enviados a este chat ahora son' in message or '<Media omitted>' in message) else message

In [114]:
# clean messages from chat
rdd_chat = rdd_chat.map(lambda message: clean_message(message))
rdd_chat = rdd_chat.filter(lambda message: message.strip())

# print messages formatted (cleaned)
rdd_chat.take(10)

['Jorge',
 '🙂',
 'Salen muy buenas',
 'digo porque ya he visto en acción',
 '//www.incolmotos-yamaha.com.co/site/productos/motos/urbanas-trabajo-y-deportivas/ybr-125',
 'O esa',
 'Wey',
 'Qué va hacer mañana?',
 'Quiubo Arley',
 'Voy salir mi familia. Luego??']